In [330]:
#Krish experimenting with ortools(idk why my code doesnt work, and idk why it does work)

In [331]:
#!pip install --upgrade ortools

In [332]:
#restart the kernel if you installed ortools and it doesnt find the module
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial
import math
import csv

import geopandas as gpd
import pandas as pd
import contextily as ctx
import pathlib
import numpy as np
import networkx
import osmnx as os
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon, LineString
import shapely
import numpy as np

In [333]:
hospitals_gdf =  gpd.read_file('Hospitals_2024.geojson')
hospitals_gdf['resources_needed'] = 500
hospitals_gdf['tag'] = 'hospital'

shelters_gdf = gpd.read_file('Shelters_2024.geojson')
shelters_gdf['resources_needed'] = 200
shelters_gdf['tag'] = 'shelter'

In [334]:
game_grid = gpd.read_file('game_grid_2024.geojson')
game_grid['point'] = [game_grid['geometry'][i].centroid for i in range(0, len(game_grid))]

In [335]:
transport_network = networkx.read_graphml('transport_network_pre.graphml')

In [336]:
nodes_df = pd.DataFrame(list(transport_network.nodes))

In [337]:
w,s,e,n = game_grid.total_bounds
airfields = os.geometries_from_bbox(n, s, e, w, tags={'aeroway': 'aerodrome'})
airfields = airfields.to_crs(game_grid.crs)
airfields['resources_needed'] = 0
airfields['tag'] = 'airfield'
airfields = airfields[['geometry', 'resources_needed','tag']]

/tmp/ipykernel_3513/2468944263.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  airfields = os.geometries_from_bbox(n, s, e, w, tags={'aeroway': 'aerodrome'})
/opt/conda/lib/python3.11/site-packages/osmnx/geometries.py:48: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return features.features_from_bbox(north, south, east, west, tags=tags)
/opt/conda/lib/python3.11/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 162 times your configured Overpass max query area size. It will automat

In [338]:
required_airfield = pd.DataFrame(data={'geometry': Point(-72.5436, 42.1991), 'resources_needed': 0, 'tag': 'airfield'}, index=[0]) #inserting mandatory airfield at beginning
airfields_gdf = pd.concat([required_airfield, airfields]).reset_index()[['geometry','resources_needed', 'tag']]
    
airfields_gdf.head()

,geometry,resources_needed,tag
0,POINT (-72.5436 42.1991),0,airfield
1,"POLYGON ((-67.0119503 44.911109, -67.0122115 4...",0,airfield
2,"POLYGON ((-67.2698893 44.6786828, -67.2694574 ...",0,airfield
3,"POLYGON ((-67.023865 44.838453, -67.030457 44....",0,airfield
4,POINT (-67.3115685 45.0273728),0,airfield


In [339]:
NUM_AIRFIELDS = 3
NUM_HOSPITALS = 10
NUM_SHELTERS = 15


VEHICLE_CAPACITY = 1200
NUM_VEHICLES = 3
SIMULATION_TIME =  72*60
NUM_AIRFIELD_POINTS = num_airfield_points
VEHICLE_SHIFT_TIME = 400

In [340]:
#construct locations gdf
#also stretches the airfields, splitting them into multiple rows based on the resources they have.

def construct_locations_gdf(airfields, hospitals, shelters):
    #creates a data matrix that stores the travel time + unloading time for every possible loctation combination, needed for or-tools
    
    hospitals_needing_supplies = hospitals.query('resources_needed > 0').sort_values(by='resources_needed', ascending=True)
    hospitals_needing_supplies['resources'] = 0
    hospitals_needing_supplies['time_replenish_amount'] = 0
    
    shelters_needing_supplies = shelters.query('resources_needed > 0').sort_values(by='resources_needed', ascending=True)
    shelters_needing_supplies['resources'] = 0
    shelters_needing_supplies['time_replenish_amount'] = 0

    airfields.loc[0:,'time_replenish_amount'] = 0 #so that starting locations of vehicles dont have replenish times
    airfields_expanded = airfields.copy()
    airfields.loc[0:,'time_replenish_amount'] = -VEHICLE_SHIFT_TIME #allows airfields to be visited by trucks to change 12-hour shifts.
    airfields_expanded.loc[0:,'resources'] = 0 #these are the starting indexes for the vehicles

    
    for index, airfield in airfields.iterrows():

        
        resources = airfield['resources']
        num_copies = math.floor(resources/VEHICLE_CAPACITY)-1 #remove one because vehicle starts loaded.

        airfield['resources'] = VEHICLE_CAPACITY
        
        
        for i in range(num_copies):
            airfields_expanded.loc[len(airfields_expanded)] = airfield

        remainder = resources % VEHICLE_CAPACITY
        airfield['resources'] = remainder
        airfields_expanded.loc[len(airfields_expanded)] = airfield

        airfield['resources'] = 0
        #while resources are listed as 0, lets add a bunch more rows to ensure there is enough time to distribute to vehicles
        for i in range(0): #set this based on how many vehicles there are and how many times each would probably need to finish a shift
            airfields_expanded.loc[len(airfields_expanded)] = airfield

        
    num_airfield_points = len(airfields_expanded)
        
    locations = pd.concat([airfields_expanded, hospitals_needing_supplies])
    locations = pd.concat([locations, shelters_needing_supplies])
    locations = locations.reset_index()[['geometry','resources_needed','resources','time_replenish_amount','tag']]

    for index, loc in airfields_gdf.iterrows():
        airfields_gdf.loc[index,'geometry'] = loc['geometry'].centroid

    return locations, num_airfield_points


In [341]:
airfields_gdf['resources'] = 3000
locations, num_airfield_points = construct_locations_gdf(airfields_gdf.iloc[:NUM_AIRFIELDS], hospitals_gdf.iloc[:NUM_HOSPITALS], shelters_gdf.iloc[:NUM_SHELTERS])

/tmp/ipykernel_3513/2756853037.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airfields.loc[0:,'time_replenish_amount'] = 0 #so that starting locations of vehicles dont have replenish times


In [342]:
locations

,geometry,resources_needed,resources,time_replenish_amount,tag
0,POINT (-72.5436 42.1991),0,0,0.0,airfield
1,"POLYGON ((-67.0119503 44.911109, -67.0122115 4...",0,0,0.0,airfield
2,"POLYGON ((-67.2698893 44.6786828, -67.2694574 ...",0,0,0.0,airfield
3,POINT (-72.5436 42.1991),0,1200,-400.0,airfield
4,POINT (-72.5436 42.1991),0,600,-400.0,airfield
5,"POLYGON ((-67.0119503 44.911109, -67.0122115 4...",0,1200,-400.0,airfield
6,"POLYGON ((-67.0119503 44.911109, -67.0122115 4...",0,600,-400.0,airfield
7,"POLYGON ((-67.2698893 44.6786828, -67.2694574 ...",0,1200,-400.0,airfield
8,"POLYGON ((-67.2698893 44.6786828, -67.2694574 ...",0,600,-400.0,airfield
9,POINT (-71.43314954499994 41.83213908400006),500,0,0.0,hospital


In [343]:
num_airfield_points

9

In [344]:
def get_closest_node(point):
    intersection = game_grid[game_grid.geometry.intersects(point)]
    
    if len(intersection) > 0:
        node_key = intersection['MGRS']
        closest_node = nodes_df[nodes_df[0].isin(node_key)].iloc[0,0]
    else:
        return None

    return closest_node

In [345]:
#shortest_paths_df = pd.DataFrame(columns=['node_1','node_2','shortest_path_time'])

In [346]:
#create data matrix from locations_gdf that stores the travel time + unloading time for every possible loctation combination, needed for or-tools

def create_data_matrix_updated(locations, shortest_paths_df):

    size = len(locations)
    
    time_matrix = np.zeros((size,size))
    
    for row in range(size):
        for col in range(size):
            row_location = locations.iloc[row]
            col_location = locations.iloc[col]

            row_loc_point = row_location['geometry']
            col_loc_point = col_location['geometry']

            row_node = get_closest_node(row_loc_point)
            col_node = get_closest_node(col_loc_point)

            if row_node == col_node:
                time_matrix[row,col] = 0
                print("same nodes!!")
    
            else:
                mask = ((shortest_paths_df['node_1'] == row_node) & (shortest_paths_df['node_2'] == col_node)) | ((shortest_paths_df['node_1'] == col_node) & (shortest_paths_df['node_2'] == row_node))
                already_exists = shortest_paths_df[mask]

                if len(already_exists) > 0:
                    time_matrix[row,col] = already_exists.iloc[0]['shortest_path_time']
                    print("utilizing dataframe!")

                else:                
                    
                    try:
                        shortest_path_time = networkx.shortest_path_length(transport_network, row_node, 
                                          col_node, weight='travel_time')
                        print("computing shortest path")
                    except:
                        shortest_path_time = 4321 #if there isn't a path between nodes, so basically algorithm will never choose this(just over 72hr)
                        
                    shortest_paths_df = pd.concat([pd.DataFrame([[row_node,col_node,shortest_path_time]], columns=shortest_paths_df.columns), shortest_paths_df])

    demand_list = list(locations['resources_needed'])

    vehicle_time_resetters_list = list(locations['time_replenish_amount'])

    shortest_paths_df = shortest_paths_df.reset_index()[['node_1','node_2','shortest_path_time']]
    
    return time_matrix.astype(np.int64).tolist(), demand_list, vehicle_time_resetters_list, shortest_paths_df

In [348]:
shortest_paths_df = pd.read_csv("shortest_paths_df.csv").reset_index()[['node_1','node_2','shortest_path_time']]
time_matrix, demand_list, vehicle_time_resetters_list, shortest_paths_df = create_data_matrix_updated(locations, shortest_paths_df)
#time_matrix, demand_list, vehicle_time_resetters_list
#old:
#282s for 85x85(25cells/s)
#220s for 85x85(32cells/s)

#updated with correct distances
#45s for 30x30(20cells/s)

same nodes!!
utilizing dataframe!
utilizing dataframe!
same nodes!!
same nodes!!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
computing shortest path
utilizing dataframe!
same nodes!!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
same nodes!!
same nodes!!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing dataframe!
utilizing datafr

In [ ]:
"""
time_matrix, demand_list, vehicle_time_resetters_list, shortest_paths_df = create_data_matrix_updated(locations, shortest_paths_df)

#ran again but with times already stored in df(shortest_paths_df)
#45s again, what
"""

In [350]:
shortest_paths_df.reset_index()[['node_1','node_2','shortest_path_time']].to_csv("shortest_paths_df.csv")

In [351]:
#in case you want to save the time_matrix and demand_list to not have to wait again.

path_ending = "_" + str(NUM_AIRFIELDS) + "a_" + str(NUM_HOSPITALS) + "h_" + str(NUM_SHELTERS) + "s.csv"

with open('time_matrix' + path_ending, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(time_matrix)

with open('demand_list' + path_ending, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(demand_list)

with open('vehicle_time_resetters_list' + path_ending, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(vehicle_time_resetters_list)

In [352]:
#Each airfield has x resources(column) to give to trucks.
#Divide by 7200, thats how many copies of the airfield to make.
#So for each airfield, you have 1 + 7200/12 rows in locations, the first one has 0 resources, others have 7200, all in the same location.

In [353]:
time_matrix

[[0,
  448,
  429,
  0,
  0,
  448,
  448,
  429,
  429,
  91,
  92,
  96,
  117,
  50,
  0,
  0,
  0,
  0,
  0,
  297,
  97,
  27,
  61,
  257,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [448,
  0,
  26,
  448,
  448,
  0,
  0,
  26,
  26,
  458,
  510,
  448,
  402,
  468,
  0,
  0,
  0,
  0,
  0,
  507,
  515,
  467,
  411,
  191,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [429,
  26,
  0,
  429,
  429,
  26,
  26,
  0,
  0,
  439,
  491,
  429,
  383,
  449,
  0,
  0,
  0,
  0,
  0,
  488,
  496,
  448,
  392,
  172,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  448,
  429,
  0,
  0,
  448,
  448,
  429,
  429,
  91,
  92,
  96,
  117,
  50,
  121,
  123,
  115,
  131,
  6,
  297,
  97,
  27,
  61,
  257,
  154,
  61,
  111,
  549,
  96,
  5,
  48,
  101,
  50,
  72],
 [0,
  448,
  429,
  0,
  0,
  448,
  448,
  429,
  429,
  91,
  92,
  96,
  117,
  50,
  121,
  123,
  115,
  131,
  6,
  297,
  97,
  27,
  61,
  257,
  154,
  61,
  111,
  549,
  96,
 

In [354]:
demand_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200]

In [355]:
vehicle_time_resetters_list

[0.0,
 0.0,
 0.0,
 -400.0,
 -400.0,
 -400.0,
 -400.0,
 -400.0,
 -400.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [356]:
def create_data_model(time_matrix, demand_list, vehicle_time_resetters_list, NUM_VEHICLES, VEHICLE_CAPACITY, VEHICLE_SHIFT_TIME):
    """Stores the data for the problem."""
    data = {}
    #print(time_matrix)

    data["time_matrix"] = time_matrix
    data["demands"] = demand_list
    
    data["vehicle_capacities"] = [VEHICLE_CAPACITY] * NUM_VEHICLES #amount of resources in vehicle
    data["num_vehicles"] = NUM_VEHICLES #number of vehicles

    data["starts"] = [0] * NUM_VEHICLES #where vehicles start
    data["ends"] = [0] * NUM_VEHICLES #where vehicles end

    data['time_per_200_units'] = 20  # 20 minutes/200 unit

    data["vehicle_times"] = [VEHICLE_SHIFT_TIME] * NUM_VEHICLES
    data["vehicle_time_resetters"] = vehicle_time_resetters_list
    print(vehicle_time_resetters_list)
    return data




def service_time(data, node):
    """Gets the service time for the specified location."""
    return int(math.ceil(abs(data['demands'][node])/200 * data['time_per_200_units']))

def travel_time(data, from_node, to_node):
    """Gets the travel times between two locations."""
    return data['time_matrix'][from_node][to_node]



def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
    """Adds capacity constraint"""
    vehicle_capacity = data["vehicle_capacities"]
    capacity = 'Capacity'
    routing.AddDimensionWithVehicleCapacity(
        demand_evaluator_index,
        VEHICLE_CAPACITY,
        vehicle_capacity,
        True,  # start cumul to zero
        capacity)

    # Add Slack for reseting to zero unload depot nodes.
    # e.g. vehicle with load 10/15 arrives at node 1 (depot unload)
    # so we have CumulVar = 10(current load) + -15(unload) + 5(slack) = 0.
    capacity_dimension = routing.GetDimensionOrDie(capacity)


def add_vehicle_time_constraints(routing, manager, data, vehicle_time_evaluator_index):
    """Adds capacity constraint"""
    vehicle_times = data["vehicle_times"]
    vehicle_time = "Vehicle Time"
    routing.AddDimensionWithVehicleCapacity(
        vehicle_time_evaluator_index,
        VEHICLE_SHIFT_TIME,
        vehicle_times,
        True,  # start cumul to zero
        vehicle_time)
    vehicle_time_dimension = routing.GetDimensionOrDie(vehicle_time)


    print(vehicle_time_dimension)






def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, from_node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(from_node)]

    return demand_evaluator




def create_time_evaluator(data):
    """Creates callback to get total times between locations."""

    _total_time = {}

    for from_node in range(len(data['time_matrix'])):
            _total_time[from_node] = {}
            for to_node in range(len(data['time_matrix'])):
                if from_node == to_node:
                    _total_time[from_node][to_node] = 0
                else:
                    _total_time[from_node][to_node] = int(
                        service_time(data, from_node) + travel_time(
                            data, from_node, to_node))

    def time_evaluator(manager, from_node, to_node):
        """Returns the total time between the two nodes"""
        
        return _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return time_evaluator











def print_solution(data, manager, routing, solution):

    
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_time = 0
    total_load = 0
    routes = []
    time = []


    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)}"
    print(dropped_nodes)

    def time_callback(from_node, to_node):
        """Creates callback to get total times between locations."""
    
        if from_node == to_node:
            return 0
            
        return int(service_time(data, from_node) + travel_time(data, from_node, to_node))


    
    for vehicle_id in range(data["num_vehicles"]):
        
        route = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_time = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index) #index, node index is the index of where the car is currently
            route.append(node_index)
            load = data["demands"][node_index]

            if load > 0:
                route_load += load
                plan_output += f" {node_index} Load({route_load})" 
            else:
                plan_output += f" {node_index} Load({route_load+load})"


            previous_index = index
            index = solution.Value(routing.NextVar(index)) #now index is index of where the car will travel
            previous_node_index = node_index
            node_index = manager.IndexToNode(index)
            """route_load += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )"""

            trip_time = time_callback(previous_node_index, node_index)
            
            route_time += trip_time
            total_time += trip_time

            plan_output += f" Time({route_time}) -> "

            #update dataframes
            
                
                
        print(route)
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Time of the route: {route_time} min\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_load += route_load
        routes.append(route)
    print(f"Total time of all routes: {total_time} min")
    print(f"Total load of all routes: {total_load}")
    print(routes)








def main():
    #print(time_matrix)
    
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(time_matrix, demand_list, vehicle_time_resetters_list, NUM_VEHICLES, VEHICLE_CAPACITY, VEHICLE_SHIFT_TIME) 

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["starts"], data["ends"])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    num_nodes = len(time_matrix)
    routing.max_callback_cache_size = 2 * num_nodes * num_nodes + NUM_VEHICLES + 1

    # Create and register a transit callback.

    # Add Capacity constraint
    demand_evaluator_index = routing.RegisterUnaryTransitCallback(partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, manager, data, demand_evaluator_index)

    # Add Vehicle Time constraint
    #vehicle_time_evaluator_index = routing.RegisterTransitCallback(partial(create_vehicle_time_evaluator(data), manager))
    #add_vehicle_time_constraints(routing, manager, data, vehicle_time_evaluator_index)
    
    time_evaluator_index = routing.RegisterTransitCallback(partial(create_time_evaluator(data), manager))

    time = "Time"
    routing.AddDimension(time_evaluator_index, 0, 100000, False, time)
    time_dimension = routing.GetDimensionOrDie(time)
    time_dimension
    
    routing.SetArcCostEvaluatorOfAllVehicles(time_evaluator_index)



    # Add Capacity constraint.
    def vt1_callback(from_index, to_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        time = int(travel_time(data, from_node, to_node) + service_time(data, from_node))
        if from_index in range(NUM_AIRFIELD_POINTS) & data["vehicle_time_resetters"][from_index] == 0:
            data["vehicle_time_resetters"][from_index] = -VEHICLE_SHIFT_TIME
            

        return time + data["vehicle_time_resetters"][from_index]
    
    

    vt1_callback_index = routing.RegisterTransitCallback(vt1_callback)
    routing.AddDimensionWithVehicleCapacity(
        vt1_callback_index,
        0,  # null capacity slack
        data["vehicle_times"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Vehicle Times Left1",
    )



    #penalty for skipping nodes
    penalty = 100000000 #big penalty to discourage skipping locations
    for node in range(NUM_AIRFIELD_POINTS, len(data["time_matrix"])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    #no penalty for skipping airbases(to maximize resources distributed
    penalty = 0 #big penalty to discourage skipping locations
    for node in range(0, NUM_AIRFIELD_POINTS):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)


    
    #set max time for simulation
    for vehicle_id in range(data['num_vehicles']):
        time_dimension.SetSpanUpperBoundForVehicle(SIMULATION_TIME, vehicle_id) 
    
    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(5)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
         print_solution(data, manager, routing, solution)

    else:
        print("No solution")


if __name__ == "__main__":
    main()

[0.0, 0.0, 0.0, -400.0, -400.0, -400.0, -400.0, -400.0, -400.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Objective: 900000000
Dropped nodes: 9 10 11 12 13 14 15 16 17
[0, 21, 20, 19, 18]
Route for vehicle 0:
 0 Load(0) Time(27) ->  21 Load(200) Time(117) ->  20 Load(400) Time(531) ->  19 Load(600) Time(852) ->  18 Load(1100) Time(908) ->  0 Load(1100)
Time of the route: 908 min
Load of the route: 1100

[0, 27, 26, 25, 24, 23, 22]
Route for vehicle 1:
 0 Load(0) Time(0) ->  27 Load(200) Time(537) ->  26 Load(400) Time(650) ->  25 Load(600) Time(885) ->  24 Load(800) Time(1209) ->  23 Load(1000) Time(1449) ->  22 Load(1200) Time(1530) ->  0 Load(1200)
Time of the route: 1530 min
Load of the route: 1200

[0, 33, 32, 31, 30, 29, 28, 8, 7, 6, 5, 4, 3, 2, 1]
Route for vehicle 2:
 0 Load(0) Time(0) ->  33 Load(200) Time(83) ->  32 Load(400) Time(154) ->  31 Load(600) Time(227) ->  30 Load(800) Time(300) ->  29

In [ ]:
def create_data_matrix_better(locations, source_nodes_all_paths_df):

    size = len(locations)
    
    time_matrix = np.zeros((size,size))

    #store list of nodes that you've done this for
    #get_all_location_nodes() to iterate over locations, keep duplicates
    #iterate over these nodes
    #compute single_source_shortest_path_length
    #convert to list, add to df along with node name
    #subset with only the nodes from get_all_location_nodes
    #add to matrix row

    
    
    for row in range(size):
        for col in range(size):
            row_location = locations.iloc[row]
            col_location = locations.iloc[col]

            row_loc_point = row_location['geometry']
            col_loc_point = col_location['geometry']

            row_node = get_closest_node(row_loc_point)
            col_node = get_closest_node(col_loc_point)

            if row_node == col_node:
                time_matrix[row,col] = 0
                print("same nodes!!")
    
            else:
                mask = ((shortest_paths_df['node_1'] == row_node) & (shortest_paths_df['node_2'] == col_node)) | ((shortest_paths_df['node_1'] == col_node) & (shortest_paths_df['node_2'] == row_node))
                already_exists = shortest_paths_df[mask]

                if len(already_exists) > 0:
                    time_matrix[row,col] = already_exists.iloc[0]['shortest_path_time']
                    print("utilizing dataframe!")

                else:                
                    
                    try:
                        shortest_path_time = networkx.shortest_path_length(transport_network, row_node, 
                                          col_node, weight='travel_time')
                        print("computing shortest path")
                    except:
                        shortest_path_time = 4321 #if there isn't a path between nodes, so basically algorithm will never choose this(just over 72hr)
                        
                    shortest_paths_df = pd.concat([pd.DataFrame([[row_node,col_node,shortest_path_time]], columns=shortest_paths_df.columns), shortest_paths_df])

    demand_list = list(locations['resources_needed'])

    vehicle_time_resetters_list = list(locations['time_replenish_amount'])

    shortest_paths_df = shortest_paths_df.reset_index()[['node_1','node_2','shortest_path_time']]
    
    return time_matrix.astype(np.int64).tolist(), demand_list, vehicle_time_resetters_list, shortest_paths_df